In [ ]:
%pip uninstall Pearl -y
%rm -rf Pearl
!git clone https://github.com/facebookresearch/Pearl.git
%cd Pearl
%pip install -e .
%cd ..

Cloning into 'Pearl'...
remote: Enumerating objects: 4364, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 4364 (delta 180), reused 175 (delta 163), pack-reused 4155
Receiving objects: 100% (4364/4364), 1.36 MiB | 7.89 MiB/s, done.
Resolving deltas: 100% (2921/2921), done.
/content/Pearl
Obtaining file:///content/Pearl
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 27.6 MB/s eta 0:00:00
  Installing build dependencies 

In [ ]:
from pearl.neural_networks.common.value_networks import EnsembleQValueNetwork
from pearl.replay_buffers.sequential_decision_making.bootstrap_replay_buffer import BootstrapReplayBuffer
from pearl.policy_learners.sequential_decision_making.bootstrapped_dqn import BootstrappedDQN
from pearl.utils.functional_utils.experimentation.set_seed import set_seed
from pearl.action_representation_modules.identity_action_representation_module import IdentityActionRepresentationModule
from pearl.history_summarization_modules.lstm_history_summarization_module import LSTMHistorySummarizationModule
from pearl.policy_learners.sequential_decision_making.deep_q_learning import DeepQLearning
from pearl.replay_buffers.sequential_decision_making.fifo_off_policy_replay_buffer import FIFOOffPolicyReplayBuffer
from pearl.utils.functional_utils.train_and_eval.online_learning import online_learning
from pearl.pearl_agent import PearlAgent
from env_model import SequenceClassificationModel
from env import RecEnv
import torch
import matplotlib.pyplot as plt
import numpy as np

set_seed(0)

SyntaxError: ignored

In [ ]:
# load environment
device = torch.device("cpu")
model = SequenceClassificationModel(100).to(device)
model.load_state_dict(torch.load("env_model_state_dict.pt"))
actions = torch.load("news_embedding_small.pt")
env = RecEnv(list(actions.values())[:100], model)
observation, action_space = env.reset()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# create a pearl agent

action_representation_module = IdentityActionRepresentationModule(
    max_number_actions=action_space.n,
    representation_dim=action_space.action_dim,
)

# DQN-vanilla
agent = PearlAgent(
    policy_learner=DeepQLearning(
        state_dim=1,
        action_space=action_space,
        hidden_dims=[64, 64],
        training_rounds=1,
        action_representation_module=action_representation_module,
    ),
    replay_buffer=FIFOOffPolicyReplayBuffer(100_000),
    device_id=-1,
)

In [ ]:
# experiment code
number_of_steps = 100000
record_period = 400

info = online_learning(
    agent=agent,
    env=env,
    number_of_steps=number_of_steps,
    print_every_x_steps=1000,
    record_period=record_period,
    learn_after_episode=True,
)
torch.save(info["return"], "DQN-return.pt")
plt.plot(record_period * np.arange(len(info["return"])), info["return"], label="DQN")
plt.legend()
plt.show()

episode 50, step 1000, agent=PearlAgent with DeepQLearning, FIFOOffPolicyReplayBuffer, env=<pearl_neurips_demo.env.RecEnv object at 0x7a0809b5c280>
return: 2.0
episode 100, step 2000, agent=PearlAgent with DeepQLearning, FIFOOffPolicyReplayBuffer, env=<pearl_neurips_demo.env.RecEnv object at 0x7a0809b5c280>
return: 1.0
episode 150, step 3000, agent=PearlAgent with DeepQLearning, FIFOOffPolicyReplayBuffer, env=<pearl_neurips_demo.env.RecEnv object at 0x7a0809b5c280>
return: 5.0
episode 200, step 4000, agent=PearlAgent with DeepQLearning, FIFOOffPolicyReplayBuffer, env=<pearl_neurips_demo.env.RecEnv object at 0x7a0809b5c280>
return: 1.0
episode 250, step 5000, agent=PearlAgent with DeepQLearning, FIFOOffPolicyReplayBuffer, env=<pearl_neurips_demo.env.RecEnv object at 0x7a0809b5c280>
return: 1.0
episode 300, step 6000, agent=PearlAgent with DeepQLearning, FIFOOffPolicyReplayBuffer, env=<pearl_neurips_demo.env.RecEnv object at 0x7a0809b5c280>
return: 4.0
episode 350, step 7000, agent=Pearl

In [ ]:
# Add a LSTM history summarization module

agent = PearlAgent(
    policy_learner=DeepQLearning(
        state_dim=128,
        action_space=action_space,
        hidden_dims=[64, 64],
        training_rounds=5,
        action_representation_module=action_representation_module,
    ),
    history_summarization_module=LSTMHistorySummarizationModule(
        observation_dim=1,
        action_dim=100,
        hidden_dim=128,
        history_length=8,
    ),
    replay_buffer=FIFOOffPolicyReplayBuffer(100_000),
    device_id=-1,
)

info = online_learning(
    agent=agent,
    env=env,
    number_of_steps=number_of_steps,
    print_every_x_steps=1000,
    record_period=record_period,
    learn_after_episode=True,
)
torch.save(info["return"], "DQN-LSTM-return.pt")
plt.plot(record_period * np.arange(len(info["return"])), info["return"], label="DQN-LSTM")
plt.legend()
plt.show()

episode 50, step 1000, agent=PearlAgent with DeepQLearning, LSTMHistorySummarizationModule(
  (lstm): LSTM(101, 128, num_layers=2, batch_first=True)
), FIFOOffPolicyReplayBuffer, env=<env.RecEnv object at 0x7db345bba200>
return: 3.0


KeyboardInterrupt: ignored

In [ ]:
# Better exploration with BootstrappedDQN-LSTM

agent = PearlAgent(
    policy_learner=BootstrappedDQN(
        q_ensemble_network=EnsembleQValueNetwork(
            state_dim=128,
            action_dim=100,
            ensemble_size=10,
            output_dim=1,
            hidden_dims=[64, 64],
            prior_scale=0.3,
        ),
        action_space=action_space,
        training_rounds=50,
        action_representation_module=action_representation_module,
    ),
    history_summarization_module=LSTMHistorySummarizationModule(
        observation_dim=1,
        action_dim=100,
        hidden_dim=128,
        history_length=8,
    ),
    replay_buffer=BootstrapReplayBuffer(100_000, 1.0, 10),
    device_id=-1,
)

info = online_learning(
    agent=agent,
    env=env,
    number_of_steps=number_of_steps,
    print_every_x_steps=100,
    record_period=record_period,
    learn_after_episode=True,
)
torch.save(info["return"], "BootstrappedDQN-LSTM-return.pt")
plt.plot(record_period * np.arange(len(info["return"])), info["return"], label="BootstrappedDQN-LSTM")
plt.legend()
plt.show()

episode 5, step 100, agent=PearlAgent with BootstrappedDQN, LSTMHistorySummarizationModule(
  (lstm): LSTM(101, 128, num_layers=2, batch_first=True)
), BootstrapReplayBuffer, env=<env.RecEnv object at 0x7db345bba200>
return: 2.0
episode 10, step 200, agent=PearlAgent with BootstrappedDQN, LSTMHistorySummarizationModule(
  (lstm): LSTM(101, 128, num_layers=2, batch_first=True)
), BootstrapReplayBuffer, env=<env.RecEnv object at 0x7db345bba200>
return: 1.0
episode 15, step 300, agent=PearlAgent with BootstrappedDQN, LSTMHistorySummarizationModule(
  (lstm): LSTM(101, 128, num_layers=2, batch_first=True)
), BootstrapReplayBuffer, env=<env.RecEnv object at 0x7db345bba200>
return: 2.0


KeyboardInterrupt: ignored